# Extracting images for each embryo

Even though the FOV contains several embryos, there is a considerable amount of background pixels that can be ignored.
Slicing the raw image into individual embryos offers significant memory reduction (average 80% for the images processed so far) and allows for easier processing downstream, since individual movies easily fit the RAM of a regular PC.

The first step is to calculate the boundaries of each embryo.
The first 10 frames of the movie are used to calculate these boundaries.
Using a low threshold (triangle method) we binarize the image and label each one of the embryos.

Once we have the boundaries, we open the entire raw data as a memmap, and extract each embryo, splitting channels 1 (active) and 2 (structural).

Embryos will be numbered in column-major order: top left corner receives 1, then second embryo is the one immediately below, etc.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

from pasnascope import slice_img

In [ ]:
# Provide absolute paths for the image and to a directory where the output 
# movies will be saved.
img_path = '/home/cdp58/Documents/raw_images/pasnascope/20240307.tif'

boundaries = slice_img.calculate_slice_coordinates(img_path,n_cols=5, thres_adjust=-10)
img = slice_img.get_first_image_from_mmap(img_path)

# Visualize the slices before parsing the data:
# The img is equalized just to display the entire embryo
fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.set_axis_off()
ax.imshow(img)

rect_coords = [slice_img.boundary_to_rect_coords(b, img.shape) for b in boundaries]
recs = [Rectangle((y,x), w, h) for (x,y,w,h) in rect_coords]

pc = PatchCollection(recs, color='red', alpha=0.7, linewidth=1, facecolor='none')
ax.add_collection(pc)

plt.show()

After inspecting the slices that will be generated, change `should_process` to `True`, so that the image is actually processed.

In [ ]:
experiment_name = '20240307'
root_dir = Path.cwd().parent
dest = root_dir.joinpath('data', experiment_name, 'embs')
should_process = False

if should_process:
    slice_img.cut_movies(boundaries, img_path, dest, overwrite=False)